In [22]:
#import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline
from datetime import datetime

In [23]:
df = pd.read_excel("19Ma.xlsx",'Data')

In [24]:
df.replace({False: 0, True: 1}, inplace=True)

In [25]:
#generating new columns based on logic

df['T_S06_straightline'] = ((df['T_S06_1']==df['T_S06_2'])&(df['T_S06_2']==df['T_S06_3']) & 
                                                (df['T_S06_3']==df['T_S06_4']) & (df['T_S06_4']==df['T_S06_5']) & 
                                                (df['T_S06_5']==df['T_S06_6']))

df['S06_Inconsistency_1'] = ((df['T_S06_2']==1) & (df['T_S06_3']==1))

#df['S06_Inconsistency_2'] = ((df['T_S06_3']==1) & (df['T_S06_6']==1))

df['B_02 Incositency'] = ((df['B02']==99) & ((df['B07_O1']==4) | (df['B07_O2']==4) | (df['B07_O3']==4) | (df['B07_O4']==4) |
                                             (df['B07_O5']==4) | (df['B07_O6']==4) | (df['B07_O7']==4) | (df['B07_O8']==4)
                                             | (df['B07_O9']==4)))


df['C02_straightline'] = ((df['T_C02_1']==df['T_C02_2'])&(df['T_C02_2']==df['T_C02_3']) & 
                                                (df['T_C02_3']==df['T_C02_4']) & (df['T_C02_4']==df['T_C02_5']) & 
                                                (df['T_C02_5']==df['T_C02_6']) & (df['T_C02_6']==df['T_C02_7']) )

df['C02_Inconsistency'] = ((df['T_C02_1']==1) & ((df['T_C02_2']==1)|(df['T_C02_3']==1)))

In [26]:
df11 = df[['Srvyr', 'VStart', 'VEnd']]

df11['Date'] = df['VStart'].dt.date
df11['V_s_time'] = df['VStart'].dt.time
df11['V_E_time'] = df['VEnd'].dt.time
df11 = df11.drop(['VStart','VEnd'],1)

<ipython-input-26-1e2328a59ee8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['Date'] = df['VStart'].dt.date
<ipython-input-26-1e2328a59ee8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['V_s_time'] = df['VStart'].dt.time
<ipython-input-26-1e2328a59ee8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [27]:
minTime=df11.groupby(['Date','Srvyr'])['V_s_time'].min()
maxTime=df11.groupby(['Date','Srvyr'])['V_E_time'].max()
numInterview=df11.groupby(['Date','Srvyr'])['V_E_time'].count()

df1 = minTime.reset_index(name 
                         = 'Start Time')
df2 = maxTime.reset_index(name 
                         = 'End Time')
df3 = numInterview.reset_index(name 
                         = 'Total Interview')


df4=(pd.to_timedelta(df2['End Time'].astype(str)) - 
                             pd.to_timedelta(df1['Start Time'].astype(str))).astype(str).map(lambda x: x[7:])


df2['Total_Time']=df4

dfs = [df1, df2, df3]

import functools as ft
dfs1 = ft.reduce(lambda left, right: pd.merge(left, right, on=['Date','Srvyr']), dfs)

In [28]:
dfs1

,Date,Srvyr,Start Time,End Time,Total_Time,Total Interview
0,2022-05-17,PN.003,09:37:35,12:53:08,03:15:33,2
1,2022-05-17,PN.004,08:15:39,13:12:40,04:57:01,6
2,2022-05-17,PN.008,07:43:03,11:54:29,04:11:26,3
3,2022-05-17,PN.010,08:56:24,12:32:52,03:36:28,4
4,2022-05-17,PN.018,09:08:50,13:26:02,04:17:12,6
5,2022-05-17,PN.020,09:04:45,13:42:33,04:37:48,9


In [29]:
g1 = round(df.groupby('Srvyr')['FlaggedByPartialRacing'].mean()*100).astype(int)
g2 = round(df.groupby('Srvyr')['FlaggedByBacktracking'].mean()*100).astype(int)
g3 = round(df.groupby('Srvyr')['FlaggedByQuestionTakingTooLong'].mean()*100).astype(int)
g4 = round(df.groupby('Srvyr')['FlaggedByClockChanged'].mean()*100).astype(int)
g5 = round(df.groupby('Srvyr')['ClockChangedMidRun'].mean()*100).astype(int)
g6 = round(df.groupby('Srvyr')['FlagsByNoGps'].mean()*100).astype(int)
g7 = round(df.groupby('Srvyr')['FlagsByFakeGPS'].mean()*100).astype(int)
g8 = round(df.groupby('Srvyr')['T_S06_straightline'].mean()*100).astype(int)
g9 = round(df.groupby('Srvyr')['B_02 Incositency'].mean()*100).astype(int)
g10 = round(df.groupby('Srvyr')['C02_straightline'].mean()*100).astype(int)
g11 = round(df.groupby('Srvyr')['C02_Inconsistency'].mean()*100).astype(int)


d1 = g1.reset_index(name = '%FlaggedByPartialRacing')
d2 = g2.reset_index(name = '%FlaggedByBacktracking')
d3 = g3.reset_index(name = '%FlaggedByQuestionTakingTooLong')
d4 = g4.reset_index(name = '%FlaggedByClockChanged')
d5 = g5.reset_index(name = '%ClockChangedMidRun')
d6 = g6.reset_index(name = '%FlagsByNoGps')
d7 = g7.reset_index(name = '%FlagsByFakeGPS')
d8 = g8.reset_index(name = '%T_S06_straightline')
d9 = g9.reset_index(name = '%B_02 Incositency')
d10 = g10.reset_index(name = '%C02_straightline')
d11 = g11.reset_index(name = '%C02_Inconsistency')



dfs2 = [d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11]

import functools as ft
dfs3 = ft.reduce(lambda left, right: pd.merge(left, right, on='Srvyr'), dfs2)


In [31]:
dfs1

,Date,Srvyr,Start Time,End Time,Total_Time,Total Interview
0,2022-05-17,PN.003,09:37:35,12:53:08,03:15:33,2
1,2022-05-17,PN.004,08:15:39,13:12:40,04:57:01,6
2,2022-05-17,PN.008,07:43:03,11:54:29,04:11:26,3
3,2022-05-17,PN.010,08:56:24,12:32:52,03:36:28,4
4,2022-05-17,PN.018,09:08:50,13:26:02,04:17:12,6
5,2022-05-17,PN.020,09:04:45,13:42:33,04:37:48,9


In [32]:
dfs3

,Srvyr,%FlaggedByPartialRacing,%FlaggedByBacktracking,%FlaggedByQuestionTakingTooLong,%FlaggedByClockChanged,%ClockChangedMidRun,%FlagsByNoGps,%FlagsByFakeGPS,%T_S06_straightline,%B_02 Incositency,%C02_straightline,%C02_Inconsistency
0,PN.003,0,0,0,0,0,0,0,0,0,0,0
1,PN.004,0,0,0,0,0,0,0,17,0,0,33
2,PN.008,0,0,0,0,0,0,0,0,0,33,67
3,PN.010,0,0,0,0,0,0,0,0,0,25,50
4,PN.018,0,0,0,0,0,0,0,0,0,33,67
5,PN.020,0,0,0,0,0,0,0,0,0,11,56


In [40]:
import pandas as pd
final_automate = pd.merge(dfs3, dfs1, on ='Srvyr', how = 'left')

In [41]:
final_automate

,Srvyr,%FlaggedByPartialRacing,%FlaggedByBacktracking,%FlaggedByQuestionTakingTooLong,%FlaggedByClockChanged,%ClockChangedMidRun,%FlagsByNoGps,%FlagsByFakeGPS,%T_S06_straightline,%B_02 Incositency,%C02_straightline,%C02_Inconsistency,Date,Start Time,End Time,Total_Time,Total Interview
0,PN.003,0,0,0,0,0,0,0,0,0,0,0,2022-05-17,09:37:35,12:53:08,03:15:33,2
1,PN.004,0,0,0,0,0,0,0,17,0,0,33,2022-05-17,08:15:39,13:12:40,04:57:01,6
2,PN.008,0,0,0,0,0,0,0,0,0,33,67,2022-05-17,07:43:03,11:54:29,04:11:26,3
3,PN.010,0,0,0,0,0,0,0,0,0,25,50,2022-05-17,08:56:24,12:32:52,03:36:28,4
4,PN.018,0,0,0,0,0,0,0,0,0,33,67,2022-05-17,09:08:50,13:26:02,04:17:12,6
5,PN.020,0,0,0,0,0,0,0,0,0,11,56,2022-05-17,09:04:45,13:42:33,04:37:48,9


In [42]:
col_name="Date"
first_col = final_automate.pop(col_name)
final_automate.insert(0, col_name, first_col)

In [43]:
final_automate.to_excel("finaout_of_automate.xlsx")

In [2]:
import pandas as pd
df = pd.DataFrame([(1,2,3), ('foo','bar','baz'), (4,5,6)])

In [3]:
df

,0,1,2
0,1,2,3
1,foo,bar,baz
2,4,5,6


In [4]:
df.columns = df.iloc[1]

In [5]:
df

1,foo,bar,baz
0,1,2,3
1,foo,bar,baz
2,4,5,6


In [9]:
df = df.drop(df.index[1])

In [10]:
df

1,foo,bar,baz
0,1,2,3
2,4,5,6
